In [129]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import re

In [130]:
#importando dataset

df_agregado = pd.read_csv('./base_agregado.csv',delimiter=';')

#Exibi o dataset

df_agregado.head(3)                        


,Placa,Código de atribuição do veículo,Propriedade de veículo,Nome completo do transportador,Tipo de transportador,Número da sua estação,Nome da sua estação,Perfil securitário veículo,Período de validade - perfil securitário veículo,Nome de tipo de veículo,...,Data de validade de inspeção anual da carta de condução,Seguro comercial expirou,Data de vencimento do seguro de carga,Data de expiração de seguro obrigatório para acidentes de trânsito,Criador,Tempo de criação,Atualizador,Tempo de atualização,Nome do condutor,CPF
0,KXV9G53,3,Veículo de logística,NaN,NaN,621101,RIO 01,无效/Invalido,2023-02-20 11:06:41,FIORINO,...,2024-09-15,2024-09-30,NaN,2024-11-28,SYLVIO PERES LUIZ,2022-09-15 18:04:48,SYLVIO PERES LUIZ,2022-09-15 18:04:48,Lucas Aguiar de Castro,6282445760
1,RJV3B38,1,carro autônomo,SPEED RABBIT EXPRESS,Permanent rent,330000,REGIONAL SPONSOR RJ,有效/Valido,2099-01-01 00:00:00,Moto,...,2023-09-30,2023-09-30,NaN,2023-09-30,VINICIUS DE SOUZA CUSTODIO,2022-09-15 18:18:12,VINICIUS DE SOUZA CUSTODIO,2022-09-15 18:18:12,ERNANI NASCIMENTO DE LIMA,17165926747
2,Ahs6441,3,Veículo de logística,NaN,NaN,621101,RIO 01,有效/Valido,2099-01-01 00:00:00,PASSEIO,...,2024-11-30,2024-09-30,NaN,2024-09-28,MARIANA ANDRADE VENANCIO,2022-09-15 19:17:33,MARIANA ANDRADE VENANCIO,2022-09-15 19:17:33,Luciano Vagner Da Costa,11466980621


In [131]:
#Verificando as colunas existente

df_agregado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6849 entries, 0 to 6848
Data columns (total 59 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Placa                                                                6849 non-null   object 
 1   Código de atribuição do veículo                                      6849 non-null   int64  
 2   Propriedade de veículo                                               1983 non-null   object 
 3   Nome completo do transportador                                       1201 non-null   object 
 4   Tipo de transportador                                                6376 non-null   object 
 5   Número da sua estação                                                6849 non-null   int64  
 6   Nome da sua estação                                                  6842 non-null   object 
 7   Perfil

In [132]:
#Exibindo somente as colunas selecionadas
df_agregados_colunas = df_agregado.loc[:,['Placa','Nome da sua estação','Nome de tipo de veículo','Nome do condutor','CPF']]

In [133]:
#Renomeando os atributos

df_agregados_colunas.rename(columns={'Placa':'PLACA',
                                     'Nome da sua estação':'NOME_BASE',
                                     'Nome de tipo de veículo':'TIPO_VEICULO',
                                     'Nome do condutor':'AGREGADO',
                                     'CPF':'CPF_CNPJ'},inplace=True)

In [134]:
#Verificando a dimensão
df_agregados_colunas.shape

(6849, 5)

In [135]:
#excluíndo cpf duplicados

df_agregados_colunas.duplicated(subset=['CPF_CNPJ']).sum()

125

In [136]:
#Armazenando os CPF duplicados e logo apos a exclusão.
df_agregados_duplicado = df_agregados_colunas[df_agregados_colunas.duplicated(subset=['CPF_CNPJ'])]

# Excluíndo as linhas duplicadas

df_agregados_clean = df_agregados_colunas.drop_duplicates(subset=['CPF_CNPJ'])

#Exibindo as dimensão

df_agregados_clean.shape
                     

(6724, 5)

In [137]:
#Verificando os missings do datafframe

df_agregados_clean.isnull().sum()

PLACA           0
NOME_BASE       7
TIPO_VEICULO    0
AGREGADO        0
CPF_CNPJ        0
dtype: int64

In [138]:
#Verificando o que são os missings
df_agregados_clean[df_agregados_clean['NOME_BASE'].isna()]

,PLACA,NOME_BASE,TIPO_VEICULO,AGREGADO,CPF_CNPJ
3136,QUF3H47,NaN,PASSEIO,MARJORY LOUREIRO,758909764
4694,KVA3662,NaN,PASSEIO,TAC CARRO - CARLOS ALBERTO NASCIMENTO DA SILVA,16829862760
4841,KXS4B99,NaN,FIORINO,MEI CARRO - ROMULO GONÇALVES TORRES DE SOUZA,14593194784
4854,LRJ0573,NaN,PASSEIO,MEI CARRO - DANIELLE MORIEL MENEZES,9003567743
6080,MPV1186,NaN,PASSEIO,MEI CARRO - DIONES LEMOS MOTHE,5765193757
6400,QUH6D81,NaN,PASSEIO,TAC CARRO - ROBERT PIRES DUPIO,16662126799
6621,KNY4D11,NaN,PASSEIO,MEI CARRO - RAI RANGEL CRESPO,16150365719


In [139]:
#Modificando os missings para 'RJ' somente para ter uma base vinculada
df_agregados_clean['NOME_BASE'].fillna('RJ',inplace=True)

#Verificando se existe missings
df_agregados_clean.isnull().sum()


PLACA           0
NOME_BASE       0
TIPO_VEICULO    0
AGREGADO        0
CPF_CNPJ        0
dtype: int64

In [140]:
#Verificando os tidos de veículos
df_agregados_clean['TIPO_VEICULO'].unique()

array(['FIORINO', 'Moto', 'PASSEIO', 'Utilitário', 'HR-Furgão', 'Truck',
       'Camionete', 'Furgao', 'VUC', '3/4', 'Baú', 'VAN'], dtype=object)

In [141]:
# Adequando os nomes do Veículos

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('FIORINO','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('PASSEIO','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('Utilitário','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('HR-Furgão','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('Truck','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('Camionete','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('Furgao','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('Furgao','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('VUC','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('3/4','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('Baú','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('VAN','CARRO')

df_agregados_clean['TIPO_VEICULO'] = df_agregados_clean['TIPO_VEICULO'].replace('Moto','MOTO')

In [142]:
#Validando a alteração
df_agregados_clean['TIPO_VEICULO'].unique()

array(['CARRO', 'MOTO'], dtype=object)

In [143]:
#Buscando os agregados que são tipo bike e estão especificados no nome.
#.loc serve para mostrar somente o atributos especifico
#str.contains('BIKE') -> Busca por essa palavras em uma string dentro do atributos
#,(TIPO_VEICULO)] = 'BIKE' -> Muda as linha que está na pesquisa em um nome especificado 'BIKE'


df_agregados_clean.loc[df_agregados_clean['AGREGADO'].str.contains('BIKE'), 'TIPO_VEICULO'] = 'BIKE'

In [144]:
df_agregados_clean['TIPO_VEICULO'].unique()

array(['CARRO', 'MOTO', 'BIKE'], dtype=object)

In [145]:
df_agregados_clean.duplicated(subset=['CPF_CNPJ']).sum()

0

In [146]:
df_agregados_clean.shape

(6724, 5)

In [147]:
df_agregados_clean.to_csv('agregados_motorista.csv',encoding='utf-8',index=False)